## Using HHM for predicting postures

In [121]:
import numpy as np
from hmmlearn.hmm import GaussianHMM
from itertools import permutations
from scipy.linalg import norm
from hmmlearn import hmm
import math
from datetime import datetime
from numpy import genfromtxt
from sklearn import preprocessing
import hmm_rand_observations_analysis as smp

input = genfromtxt('/Users/Madhu/Documents/HMMHealth_new/A01.txt', delimiter=',',dtype='str')
input = input[np.where(input[:,1]=='020-000-033-111')]
input = input[:,(2,4,5,6,7)]
#input = input[np.where(input[:,4]=='walking')]
walk=np.where(input[:,4]=='walking')[0]
sit=np.where(input[:,4]=='sitting')[0]
lay=np.where(input[:,4]=='lying')[0]
attr=walk.tolist()
attr.extend(sit.tolist())
attr.extend(lay.tolist())
input=input[attr]


## Encoding the postures to numerical values

In [54]:
encoding = preprocessing.LabelEncoder()  
input[:, 4] = encoding.fit_transform(input[:, 4])
mapping=[]
for i in sorted(set(input[:,4].astype(float))):
    mapping.append((int(i),encoding.inverse_transform(int(i))),)
input = sorted(input, key = lambda a_entry: a_entry[0]) 
input = np.array(input)
input = input[:,(1,2,3,4)]
input = input.astype(float)
mapping

[(0, 'falling'),
 (1, 'lying'),
 (2, 'lying down'),
 (3, 'on all fours'),
 (4, 'sitting'),
 (5, 'sitting down'),
 (6, 'sitting on the ground'),
 (7, 'standing up from lying'),
 (8, 'standing up from sitting'),
 (9, 'standing up from sitting on the ground'),
 (10, 'walking')]

## Splitting data into training and testing

In [65]:
train_rows = int(math.floor(0.7* input.shape[0]))
test_rows = input.shape[0] - train_rows
    
# separate out training and testing data
trainX = input[:train_rows,:-1]
trainZ = input[:train_rows,-1]
trainZ=trainZ.astype(int)
testX = input[train_rows:,:-1]
testZ = input[train_rows:,-1]
testZ=testZ.astype(int)

print "Total    ",input.shape
print "Training ",trainX.shape," Label ",trainZ.shape
print "Testing  ",testX.shape,"  Label ",testZ.shape,"\n"
print input[:3,:]

Total     (1643, 4)
Training  (1150, 3)  Label  (1150,)
Testing   (493, 3)   Label  (493,) 

[[  4.29195356   1.78114045   1.3444953   10.        ]
 [  4.36407709   1.76769149   1.29999638  10.        ]
 [  4.24491358   1.98168194   1.30875885  10.        ]]


## Gaussian distribution over the 11 latent variables 

In [70]:
model_p = GaussianHMM(11,'diag').fit(trainX)
hidden_states = model_p.predict(trainX)
l = list(permutations(range(0, 11)))
acctrain=-1
cur_acc=0

## Transition probability from the 11 states 

In [69]:
print model_p.transmat_

[[  9.86603108e-001   1.65859181e-223   1.17246499e-209   5.19778828e-034
    7.59751102e-053   2.58446357e-066   4.00530336e-027   1.35847843e-035
    4.73477619e-021   1.72653210e-123   1.33968924e-002]
 [  1.13560283e-217   9.93048006e-001   0.00000000e+000   0.00000000e+000
    7.21269529e-069   0.00000000e+000   9.93523740e-170   6.95199369e-003
    7.27013362e-060   0.00000000e+000   1.43665395e-024]
 [  4.94577503e-279   0.00000000e+000   8.29721667e-001   0.00000000e+000
    0.00000000e+000   1.70278333e-001   5.68086558e-013   8.93738720e-178
    1.90190260e-070   0.00000000e+000   3.55330707e-123]
 [  6.95942587e-036   0.00000000e+000   0.00000000e+000   9.92576653e-001
    1.73735612e-120   1.39597390e-130   7.57829171e-054   1.44170524e-024
    5.01643944e-024   0.00000000e+000   7.42334731e-003]
 [  2.11266120e-063   4.09616962e-086   0.00000000e+000   9.02055087e-140
    9.92110370e-001   3.82823438e-177   7.81192874e-074   1.03227677e-014
    4.93971526e-028   3.74008143

## Mean of observations for each Gaussian curve

In [71]:
print model_p.means_

[[ 3.24930054  1.50626728  0.78562608]
 [ 0.98542594  2.37091013  0.39878206]
 [ 4.43054371  1.46751524 -0.09302002]
 [ 2.73118203  1.20986261  0.96970741]
 [ 2.88962179  2.53862555  0.97665749]
 [ 4.73816627  1.15454315  0.37030177]
 [ 3.95923168  1.35915821  0.73620822]
 [ 1.78882271  2.09524335  0.87331512]
 [ 3.79313212  1.74504885  1.25765655]
 [ 0.0354894   3.39953548  2.59576696]
 [ 3.03391324  1.73102324  1.03973569]]


## Covariance of observations for each Gaussian curve 

In [72]:
print model_p.covars_

[[[ 0.00428712  0.          0.        ]
  [ 0.          0.0111911   0.        ]
  [ 0.          0.          0.00479561]]

 [[ 0.00542403  0.          0.        ]
  [ 0.          0.01818258  0.        ]
  [ 0.          0.          0.00804287]]

 [[ 0.00116565  0.          0.        ]
  [ 0.          0.00054715  0.        ]
  [ 0.          0.          0.00434727]]

 [[ 0.00046551  0.          0.        ]
  [ 0.          0.00073211  0.        ]
  [ 0.          0.          0.00114454]]

 [[ 0.00571457  0.          0.        ]
  [ 0.          0.00922986  0.        ]
  [ 0.          0.          0.00619444]]

 [[ 0.0590793   0.          0.        ]
  [ 0.          0.07562659  0.        ]
  [ 0.          0.          0.06617199]]

 [[ 0.03940309  0.          0.        ]
  [ 0.          0.03800587  0.        ]
  [ 0.          0.          0.0134037 ]]

 [[ 0.16855908  0.          0.        ]
  [ 0.          0.07321271  0.        ]
  [ 0.          0.          0.07505585]]

 [[ 0.14968332  0.      

## Starting probability

In [73]:
print model_p.startprob_

[  0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   1.78818666e-96   4.30550230e-51   7.99195975e-96
   1.00000000e+00   0.00000000e+00   3.60883983e-62]


In [25]:
l_part= [ l[index] for index in range(1,100000) ]
for p in l_part:
    var_trainZ=trainZ
    for i in range(trainZ.shape[0]):
        var_trainZ[i]=p[trainZ[i]]
    cur_acc=np.mean( hidden_states == var_trainZ )
    if(cur_acc>acctrain):
        acctrain=cur_acc
        perm_m=p

In [75]:
print acctrain,hidden_states[:10],trainZ[:10]


[8 8 8 8 8 8 8 8 8 8] [10 10 10 10 10 10 10 10 10 10]


In [6]:
np.sum(model_p.transmat_,axis=1)
#np.sum(model_p.startprob_)

array([ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.])

In [100]:
t=[]
#t.extend(list(np.where(input[:,4]=='walking')))
#len(t)
type(t)

list

In [96]:
t.extend(np.where(input[:,4]=='lying'))
len(t)

2

In [88]:
t

[array([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,   10,
          11,   12,   13,   14,   15,   16,   17,   18,   19,   20,   21,
          22,   23,   24,   25,   26,   27,   28,   29,   30,   31,   32,
         184,  185,  186,  187,  188,  189,  190,  191,  192,  193,  194,
         195,  196,  380,  381,  382,  383,  384,  385,  386,  387,  388,
         389,  390,  391,  392,  393,  394,  395,  396,  397,  398,  399,
         400,  401,  402,  403,  404,  405,  406,  407,  408,  409,  410,
         411,  412,  413,  414,  415,  416,  417,  418,  604,  605,  606,
         607,  608,  609,  610,  611,  612,  613,  614,  615,  616,  617,
         618,  619,  620,  621,  622,  623,  624,  625,  626,  627,  628,
         629,  630,  631,  632,  633,  634,  635,  636,  637,  638,  639,
         640,  641,  642,  643,  644,  645,  646,  647,  823,  824,  825,
         826,  827,  828,  829,  830,  831,  832,  833,  834,  835,  836,
         837,  838,  839,  840,  841, 

In [119]:
walk=np.where(input[:,4]=='walking')[0]
sit=np.where(input[:,4]=='sitting')[0]
lay=np.where(input[:,4]=='lying')[0]
attr=walk.tolist()
print len(attr)
attr.extend(sit.tolist())
print len(attr)
attr.extend(lay.tolist())
print len(attr)

244
532
1145


In [104]:
(np.where(input[:,4]=='walking')).tolist()

AttributeError: 'tuple' object has no attribute 'tolist'

In [120]:
attr

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 184,
 185,
 186,
 187,
 188,
 189,
 190,
 191,
 192,
 193,
 194,
 195,
 196,
 380,
 381,
 382,
 383,
 384,
 385,
 386,
 387,
 388,
 389,
 390,
 391,
 392,
 393,
 394,
 395,
 396,
 397,
 398,
 399,
 400,
 401,
 402,
 403,
 404,
 405,
 406,
 407,
 408,
 409,
 410,
 411,
 412,
 413,
 414,
 415,
 416,
 417,
 418,
 604,
 605,
 606,
 607,
 608,
 609,
 610,
 611,
 612,
 613,
 614,
 615,
 616,
 617,
 618,
 619,
 620,
 621,
 622,
 623,
 624,
 625,
 626,
 627,
 628,
 629,
 630,
 631,
 632,
 633,
 634,
 635,
 636,
 637,
 638,
 639,
 640,
 641,
 642,
 643,
 644,
 645,
 646,
 647,
 823,
 824,
 825,
 826,
 827,
 828,
 829,
 830,
 831,
 832,
 833,
 834,
 835,
 836,
 837,
 838,
 839,
 840,
 841,
 842,
 843,
 844,
 845,
 846,
 847,
 848,
 849,
 850,
 851,
 852,
 853,
 854,
 855,
 856,
 857,
 858,
 859,
 860,
 861,
 862,
 863,
 864,
 865,
 1221,
 122

In [123]:
set(input[:,4])

{'lying', 'sitting', 'walking'}